In [11]:
import numpy as np
import qiskit
from qiskit.quantum_info import state_fidelity
from numpy import linalg as LA
import qib
import matplotlib.pyplot as plt
import scipy
import h5py

import sys
sys.path.append("../../src/brickwall_sparse")
from utils_sparse import construct_ising_local_term, reduce_list, X, I2, get_perms
from ansatz_sparse import ansatz_sparse
import rqcopt as oc
from scipy.sparse.linalg import expm_multiply
from qiskit.quantum_info import random_statevector
from scipy.linalg import expm
from itertools import product
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

J, h, g = (1, 0, 3)
Lx, Ly = (4, 4)
L = Lx*Ly
t = 0.125
# construct Hamiltonian
latt = qib.lattice.TriangularLattice((Lx, Ly), pbc=True)
field = qib.field.Field(qib.field.ParticleType.QUBIT, latt)
hamil = qib.IsingHamiltonian(field, J, h, g).as_matrix()

perms_1 = [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], [1, 2, 3, 0, 5, 6, 7, 4, 9, 10, 11, 8, 13, 14, 15, 12]]
perms_2 = [[0, 5, 10, 15, 3, 4, 9, 14, 2, 7, 8, 13, 1, 6, 11, 12], [5, 10, 15, 0, 4, 9, 14, 3, 7, 8, 13, 2, 6, 11, 12, 1]]
perms_3 = [[0, 4, 8, 12, 1, 5, 9, 13, 2, 6, 10, 14, 3, 7, 11, 15], [4, 8, 12, 0, 5, 9, 13, 1, 6, 10, 14, 2, 7, 11, 15, 3]]

#eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(hamil, k=100)
eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(hamil, k=10, which='SA')
idx = eigenvalues.argsort()
eigenvalues_sort = eigenvalues[idx]
eigenvectors_sort = eigenvectors[:,idx]
ground_state = eigenvectors_sort[:, 0]

perms = [perms_1]  + [perms_2] + [perms_3]

In [12]:
import h5py

with h5py.File("./results/triangularTFIM_RQCOPT_SPARSE_103_Lx4Ly4_t0.125_layers3_niter5_rS1_2hloc.hdf5", "r") as f:
    Vlist = f["Vlist"][:]

In [13]:
"""
    Test operations.
"""

state = random_statevector(2**L).data
print("Trotter error of the optimized point: ", np.linalg.norm(ansatz_sparse(Vlist, L, perms, state) - expm_multiply(
    -1j * t * hamil, state), ord=2) )

print('infidelity: ', 1-(state_fidelity(ansatz_sparse(Vlist, L, perms, state), expm_multiply(
    -1j * t * hamil, state))))

Trotter error of the optimized point:  0.08693409420347302
infidelity:  0.007543253075517065


In [15]:
sys.path.append("../../src/controlled_unitary_optimizer")
sys.path.append("../../src/brickwall_ansatz")
from optimize_3q import optimize_3q 
from utils_3q import make_controlled, random_unitary

Xlists_opt = {}
perms_qc = [[0, 1], [0, 2], [1, 2], [0, 2], [0, 1], [1, 2], [0, 2], [0, 1], [1, 2]]
#perms_qc = [[0, 1], [0, 2]]

for i in [0, 1, 2]:
    cU = make_controlled(Vlist[i])
    f_best, err_best, Glist_best = (0, 2, None)
    for _ in range(10):
        Xlist_start = [random_unitary(4) for i in range(len(perms_qc))]
        Xlist, f_iter, err_iter = optimize_3q(L, cU, Xlist_start, perms_qc, niter=1000)
        if err_iter[-1] < err_best:
            f_best, err_best, Xlist_best = (f_iter[-1], err_iter[-1], Xlist)
    print("Best f: ", f_best)
    print("Best err: ", err_best)
    Xlists_opt[i] = Xlist_best

/opt/anaconda3/envs/qc/lib/python3.10/site-packages/rqcopt/trust_region.py:40: RuntimeWarning: divide by zero encountered in scalar divide
  rho = (f(x_next) - fx) / (np.dot(grad, eta) + 0.5 * np.dot(eta, hess @ eta))


Best f:  -7.999999999999923
Best err:  1.977102301984429e-07
Best f:  -7.9999999999814655
Best err:  2.8235350181699922e-06
Best f:  -7.999999999999782
Best err:  3.120721780711942e-07


In [19]:
from qiskit import Aer, execute, transpile
from qiskit.circuit.library import CYGate, CZGate, IGate, CXGate
from qiskit.converters import circuit_to_dag
from qiskit.providers.aer.noise import NoiseModel, errors
from qiskit import Aer, execute, transpile
from scipy import sparse as sp


qc = qiskit.QuantumCircuit(L+1)


for i, V in enumerate(Vlist):
    layer = i
    if True:
        Glist = Xlists_opt[i]
        qc_3 = qiskit.QuantumCircuit(3)
        for j, G in enumerate(Glist):
            qc_3.unitary( G, (3-1-perms_qc[j][1], 3-1-perms_qc[j][0]))
    
        for perm in perms[layer]:
            for j in range(L//2):
                qc.append(qc_3.to_gate(), [L-perm[2*j]-1, L-perm[2*j+1]-1, L])



ccU_cxs_0125= []
ccU_errs_0125 = []
for t_ in [2*t, 4*t, 8*t]:
    state = random_statevector(2**L).data
    backend = Aer.get_backend("statevector_simulator")

    qc_ext2 = qiskit.QuantumCircuit(L+1)
    qc_ext2.initialize(state, [i for i in range(L)])
    qc_ext2.x(L)
    for i in range(int(t_/t)):
        qc_ext2.append(qc.to_gate(), [i for i in range(L+1)])
    backend = Aer.get_backend("statevector_simulator")
    sv2 = execute(transpile(qc_ext2), backend).result().get_statevector().data

    ket_0 = np.array([1, 0])
    ket_1 = np.array([0, 1])
    ket_0_e = np.eye(1)
    ket_1_e = np.eye(1)
    for i in range(1):
        ket_0_e = np.kron(ket_0_e, ket_0)
        ket_1_e = np.kron(ket_1_e, ket_1)
    exact_v2 = np.kron(ket_1_e, expm_multiply(-1j * t_ * hamil, state))
    err = 1-np.abs(np.vdot(sv2, exact_v2))**2
    
    noise_model = NoiseModel()
    dag = circuit_to_dag(transpile(qc_ext2, basis_gates=noise_model.basis_gates+['unitary', 'initialize', 'cx']))
    count_ops = dag.count_ops()
    print(count_ops)
    
    ccU_errs_0125.append(err)
    ccU_cxs_0125.append(count_ops['unitary'])
    print("Trace norm: ", err)

{'initialize': 1, 'sx': 2, 'unitary': 864}
Trace norm:  0.01806868230452452
{'initialize': 1, 'sx': 2, 'unitary': 1728}
Trace norm:  0.030610631523913967
{'initialize': 1, 'sx': 2, 'unitary': 3456}
Trace norm:  0.06675485313714868
